# Azure AI Agent Service Enterprise Demo

### Import Necessary Libraries
In this cell, we import all the libraries and modules required for the project.
This includes Azure AI SDKs, Gradio for UI, and custom functions.

In [1]:
import os
import re
import uuid
from datetime import datetime as pydatetime
from typing import Any, List, Dict
from dotenv import load_dotenv

# (Optional) Gradio app for UI
import gradio as gr
from gradio import ChatMessage
import base64

# Azure AI Projects
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
# from azure.ai.agents import AgentsClient
from azure.ai.agents.models import (
    AgentEventHandler,
    RunStep,
    RunStepDeltaChunk,
    ThreadMessage,
    ThreadRun,
    MessageDeltaChunk,
    BingGroundingTool,
    FilePurpose,
    FileSearchTool,
    FunctionTool,
    ToolSet,
    VectorStore,
    AzureAISearchTool,
    CodeInterpreterTool,
    MessageDeltaTextContent,
    MessageDeltaImageFileContent,
    MessageDeltaTextContentObject,
    MessageDeltaTextUrlCitationAnnotation,
    MessageRole,
    AgentThread,
    MessageTextContent,
)

# Your custom Python functions (for "fetch_datetime", etc.)
from utils.enterprise_functions import enterprise_fns

load_dotenv(dotenv_path=".env", override=True)

# REQUIRED environment variables (replace with your values or a .env file)
REQUIRED_KEYS = [
    'PROJECT_ENDPOINT',
    'MODEL_DEPLOYMENT_NAME',
    'AGENT_NAME',
    'BING_CONNECTION_NAME',
    'AZURE_SEARCH_CONNECTION_NAME',
    'AZURE_SEARCH_INDEX_NAME',
]

missing = [k for k in REQUIRED_KEYS if not os.getenv(k)]
if missing: # [] false
    raise EnvironmentError(f'Missing required env keys: {missing}')

# Authenticate (interactive fallback)
try:
    credential = DefaultAzureCredential()
except Exception:
    credential = InteractiveBrowserCredential()

print('Environment and authentication OK')

Environment and authentication OK


### Create Client and Load Azure AI Foundry
Here, we initialize the Azure AI client using DefaultAzureCredential.
This allows us to authenticate and connect to the Azure AI service.

In [2]:
# new AI Foundry Project resource endpoint / old azure ai services endpoint from the hub/project
project_client = AIProjectClient(
    credential=credential,
    endpoint=os.environ["PROJECT_ENDPOINT"],
    # api_version=os.environ["PROJECT_API_VERSION"]
)
print("project_client api version:", project_client._config.api_version)

project_client api version: 2025-05-15-preview


### Set Up Tools (BingGroundingTool, FileSearchTool)
In this step, we configure tools such as `BingGroundingTool` and `FileSearchTool`.
We check for existing connections and create or reuse vector stores for document search.

Note:
If you see the following cell has error:
```
AzureCliCredential: Please run 'az login' to set up an account
```

relogin from powershell
```powershell
az logout
az account clear
az login --tenant 00000000-0000-0000-0000-000000000000
```


In [3]:
try:
    bing_connection = project_client.connections.get(name=os.environ["BING_CONNECTION_NAME"])
    # print(f"{bing_connection}")
    conn_id = bing_connection.id
    bing_tool = BingGroundingTool(connection_id=conn_id)
    print("bing > connected")
except Exception:
    bing_tool = None
    print("bing failed > no connection found or permission issue")

## need to be wrapped inside the agents_client, close agents_client if done
FOLDER_NAME = "enterprise-data"
VECTOR_STORE_NAME = "hr-policy-vector-store"

# project_client.agents return the AgentsClient
all_vector_stores: List[VectorStore] = project_client.agents.vector_stores.list()

existing_vector_store = next(
    (store for store in all_vector_stores if store.name == VECTOR_STORE_NAME),
    None
)

vector_store_id = None
if existing_vector_store:
    vector_store_id = existing_vector_store.id
    print(f"reusing vector store > {existing_vector_store.name} (id: {existing_vector_store.id})")
else:
    # If you have local docs to upload
    import os
    if os.path.isdir(FOLDER_NAME):
        file_ids = []
        for file_name in os.listdir(FOLDER_NAME):
            file_path = os.path.join(FOLDER_NAME, file_name)
            if os.path.isfile(file_path):
                print(f"uploading > {file_name}")
                uploaded_file = project_client.agents.files.upload_and_poll(
                    file_path=file_path,
                    purpose=FilePurpose.AGENTS
                )
                file_ids.append(uploaded_file.id)

        if file_ids:
            print(f"creating vector store > from {len(file_ids)} files.")
            vector_store = project_client.agents.vector_stores.create_and_poll(
                file_ids=file_ids,
                name=VECTOR_STORE_NAME
            )
            vector_store_id = vector_store.id
            print(f"created > {vector_store.name} (id: {vector_store_id})")

file_search_tool = None
if vector_store_id:
    file_search_tool = FileSearchTool(vector_store_ids=[vector_store_id])

bing > connected
reusing vector store > hr-policy-vector-store (id: vs_AMBfvCNoWCerPKW15891APxf)


In [4]:
# Get the connection ID for your Azure AI Search resource
try:
    aisearch_connections = project_client.connections.list()
    idx_conn_id = next(
        c.id for c in aisearch_connections if c.name == os.environ.get("AZURE_SEARCH_CONNECTION_NAME")
    )

    # Initialize Azure AI Search tool for direct index access
    search_tool = AzureAISearchTool(
        index_connection_id=idx_conn_id,
        index_name=os.environ.get("AZURE_SEARCH_INDEX_NAME")
    )
    print("azure ai search > connected directly to index")
except Exception as e:
    search_tool = None
    print(f"azure ai search > skipped (no connection configured): {str(e)}")

azure ai search > connected directly to index


### Combine All Tools into a ToolSet
This step creates a custom `ToolSet` that includes all the tools configured earlier.
It also adds a `LoggingToolSet` subclass to log the inputs and outputs of function calls.

In [5]:
class LoggingToolSet(ToolSet):
    def execute_tool_calls(self, tool_calls: List[Any]) -> List[dict]:
        """
        Execute the upstream calls, printing only two lines per function:
        1) The function name + its input arguments
        2) The function name + its output result
        """

        # For each function call, print the input arguments
        for c in tool_calls:
            if hasattr(c, "function") and c.function:
                fn_name = c.function.name
                fn_args = c.function.arguments
                print(f"{fn_name} inputs > {fn_args} (id:{c.id})")

        # Execute the tool calls (superclass logic)
        raw_outputs = super().execute_tool_calls(tool_calls)

        # Print the output of each function call
        for item in raw_outputs:
            print(f"output > {item['output']}")

        return raw_outputs

# need an empty toolset to add tools
custom_functions = FunctionTool(enterprise_fns)

toolset = LoggingToolSet()

# if file_search_tool:
#      toolset.add(file_search_tool)
if bing_tool:
    toolset.add(bing_tool)
toolset.add(custom_functions)
if search_tool:
    toolset.add(search_tool)

for tool in toolset._tools:
    tool_name = tool.__class__.__name__
    print(f"tool > {tool_name}")
    for definition in tool.definitions:
        if hasattr(definition, "function"):
            fn = definition.function
            print(f"{fn.name} > {fn.description}")
        else:
            pass

tool > BingGroundingTool
tool > FunctionTool
tool > AzureAISearchTool


In [6]:
### (Optional) Direct Azure AI Search Integration - No AI Search in this example
## Skip this cell if you're using the default File Search Tool vector store approach

# (Optional) Direct Azure AI Search Integration
# If the default File Search Tool is available, we add it and skip the Azure AI Search integration.
# if any(tool.__class__.__name__ == "FileSearchTool" for tool in toolset._tools):
#     print("file_search tool exists > skipping ai_search tool add")
# else:
#     try:
#         # Get the connection ID for your Azure AI Search resource
#         connections = project_client.connections.list()
#         conn_id = next(
#             c.id for c in connections if c.name == os.environ.get("AZURE_SEARCH_CONNECTION_NAME")
#         )
        
#         # Initialize Azure AI Search tool for direct index access
#         from azure.ai.projects.models import AzureAISearchTool
#         search_tool = AzureAISearchTool(
#             index_connection_id=conn_id,
#             index_name=os.environ.get("AZURE_SEARCH_INDEX_NAME")
#         )
        
#         # Add the Azure AI Search tool to our toolset
#         toolset.add(search_tool)
#         print("azure ai search > connected directly to index")
        
#         # Verify the tool was added by iterating through the toolset
#         for tool in toolset._tools:
#             tool_name = tool.__class__.__name__
#             print(f"tool > {tool_name}")
#             for definition in tool.definitions:
#                 if hasattr(definition, "function"):
#                     fn = definition.function
#                     print(f"{fn.name} > {fn.description}")
#                 else:
#                     pass

#     except Exception as e:
#         print(f"azure ai search > skipped (no connection configured): {str(e)}")


### Create or Reuse the Enterprise Agent
In this step, we create a new enterprise agent or reuse an existing one.
The agent is configured with a model, instructions, and the toolset from the previous step.

Note:
* You will need to delete the previous agent, while recreate

In [7]:
AGENT_NAME = os.environ.get("AGENT_NAME", "my-enterprise-agent")
found_agent = None
all_agents_list = project_client.agents.list_agents()
for a in all_agents_list:
    if a.name == AGENT_NAME:
        found_agent = a
        break

model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4.1-mini")

# "fetch_datetime": "🕒 fetching datetime",
# "file_search": "📄 searching docs",
# "bing_grounding": "🔍 searching bing",
# "azure_ai_search": "🔎 ai search private index",

instructions = (
    "You are a helpful enterprise assistant at Contoso. "
    "You have access to following tools. \n\n"
    "## Tools:\n"
    " * azure_ai_search: get information about company financial reportings\n"
    " * file_search: get informaton about company Human Resource documents\n"
    " * bing_grounding: get information about latest news from the public web.\n"
    " * fetch_datetime: to get the current date/time.\n"
    "\n"
    "## Instructions:\n"
    "You can use the all the tools to answer questions\n"
    "\n"
    "## Guidelines:\n"  
    # "Make a plan and you should always try to answer the question using the tools available in the plan. "
    # "List the tools you want to use first. Then execute the plan step by step, and execute the tools in the order you listed them. "  
    # "If you cannot find the answer using the use ai_search and file_search at the same time.\n"
    # "If you cannot find the answer using the file_search first, then use ai_search instead. Lastly you can use the BingGroundingTool to search the web to get latest information. "
    # "You can iterate on the plan and refine your answer using the tools available. "
    "Provide well-structured and professional answers. "
)

# override for testing purposes
# instructions = ("Allways use all the tools available to answer the question. ")

project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4)

if found_agent:
    # print(found_agent)
    # Update the existing agent to use new tools
    agent = project_client.agents.update_agent(
        agent_id=found_agent.id,
        model=model_name,
        instructions=instructions,
        toolset=toolset,
    )
    project_client.agents.enable_auto_function_calls(tools=toolset) 
    print(f"reusing agent > {agent.name} (id: {agent.id})")
else:
    agent = project_client.agents.create_agent(
        model=model_name,
        name=AGENT_NAME,
        instructions=instructions,
        toolset=toolset,
    )
    print(f"creating agent > {agent.name} (id: {agent.id})")

reusing agent > eval-enterprise-agent (id: asst_kDH574eDU34CbTDoprzD1rNy)


### Create a Conversation Thread
In this step, we create a new conversation thread for the enterprise agent.
Threads are used to manage and track conversations with the agent.

In [8]:
thread = project_client.agents.threads.create()
print(f"thread > created (id: {thread.id})")

thread > created (id: thread_lUfq84gJ2eywFFCJlRc7jUqy)


## (Optional) Manipulate the thread

In [9]:
thread_whole_conversation = project_client.agents.threads.create()
print(f"thread > created (id: {thread_whole_conversation.id})")

# user_prompt_1 = "tell me about the latest news about microsoft azure"
user_prompt_2 = "summarize siemens fiscal report 2024 from my company source"

def generate_conversation_thread_single(current_thread: AgentThread, user_prompt: str):
    msg = project_client.agents.messages.create(
        thread_id=current_thread.id,
        role=MessageRole.USER,
        content=user_prompt
    )
    print(f"message > created (id: {msg.id})")

def display_message(thread_message: ThreadMessage):
    for agent_message in thread_message.content:
        if isinstance(agent_message, MessageTextContent):
            agent_msg_obj = agent_message.text
            annotations = agent_msg_obj.get("annotations", None)
            print(f"agent message text > {agent_msg_obj.value} (id: {thread_message.id})")
            # get the grounding information from the agent message
            if annotations and isinstance(annotations, list) and len(annotations) > 0:
                print(f"\nGrounding > {annotations}")

# def generate_conversation_thread_multi(current_thread: AgentThread, user_prompt: str):
#     msg = project_client.agents.messages.create(
#         thread_id=current_thread.id,
#         role=MessageRole.USER,
#         content=user_prompt
#     )
#     print(f"message > created (id: {msg.id})")

#     msg = project_client.agents.messages.create(
#         thread_id=current_thread.id,
#         role=MessageRole.AGENT,
#         content="Agent1: how can I help you today?"
#     )

#     msg = project_client.agents.messages.create(
#         thread_id=current_thread.id,
#         role=MessageRole.AGENT,
#         content="Agent2: What's up?"
#     )

# put the conversation history single agent to the thread
generate_conversation_thread_single(current_thread=thread_whole_conversation, user_prompt=user_prompt_2)

# run agent based on the thread conversation history
run = project_client.agents.runs.create_and_process(thread_id=thread_whole_conversation.id, agent_id=agent.id, temperature=0.1)

thread > created (id: thread_qiiGzMpEIKBk9nM5s2TZlsch)
message > created (id: msg_TvyEsQvgiKjM1STK9MLV34yK)


In [10]:
agent_message_annotation: ThreadMessage = project_client.agents.messages.get_last_message_by_role(thread_id = thread_whole_conversation.id, role=MessageRole.AGENT)

display_message(agent_message_annotation)

agent message text > The Siemens fiscal report for 2024 highlights the following key points:

1. Financial Performance:
- Siemens achieved a net income of €9.0 billion, reaching a historic high.
- Basic earnings per share (EPS) increased to €10.53, with EPS pre-PPA (Purchase Price Allocation) at €11.15.
- Return on capital employed (ROCE) rose to 19.1%, within the target range of 15% to 20%.
- The ratio of industrial net debt to EBITDA was 0.7, well below the forecasted maximum of 1.5.
- Free cash flow was €9.5 billion, slightly below the record €10.0 billion in 2023.

2. Segment Performance:
- Digital Industries maintained the highest profit margin among industrial businesses at 18.9%, though it declined year-over-year.
- Mobility improved its profit margin to 8.9%.
- Siemens Financial Services (SFS) saw significant earnings growth, with a return on equity after tax of 17.6%.

3. Economic Outlook and Risks:
- Moderate global economic growth is expected in fiscal 2025, with ongoing geo

### Testing Bing grounding tool

In [11]:
user_prompt_1 = "tell me about the latest news about microsoft azure"

bing_grounding_thread = project_client.agents.threads.create()
print(f"thread > created (id: {bing_grounding_thread.id})")

# put the conversation history single agent to the thread
generate_conversation_thread_single(current_thread=bing_grounding_thread, user_prompt=user_prompt_1)

# run agent based on the thread conversation history
run = project_client.agents.runs.create_and_process(thread_id=bing_grounding_thread.id, agent_id=agent.id, temperature=0.1)

thread > created (id: thread_T3KletNQ85AyBfkvNFnqpWsk)
message > created (id: msg_1m2blIc2PDCPtqKwnLXCW08X)


In [12]:
bing_grounding_agent_thread_message: ThreadMessage = project_client.agents.messages.get_last_message_by_role(thread_id = bing_grounding_thread.id, role=MessageRole.AGENT)

display_message(bing_grounding_agent_thread_message)

agent message text > The latest news about Microsoft Azure highlights several key advancements and updates:

1. At Build 2025, Scott Guthrie, executive vice president of Microsoft Cloud + AI, showcased the latest Azure Boost innovations. These include new network-optimized virtual machines (VMs) such as the Dnsv6 and Ensv6 series, which enhance network performance and optimization.

2. Microsoft unveiled Azure AI Foundry, a new AI application creation and management platform. This platform integrates Azure AI Studio and connects with popular integrated development environments (IDEs) like Visual Studio to help businesses consolidate AI across their technology stack and drive AI adoption.

3. During Microsoft Ignite, over 100 updates were announced related to generative AI infrastructure breakthroughs, new tools, and advancements in adoption, productivity, and security. These updates are designed to be transformative for developers and startups building with AI on Microsoft Azure.

4. A

### Testing the Content Filter with Block List

In [13]:
block_list_thread = project_client.agents.threads.create()
print(f"thread > created (id: {block_list_thread.id})")

# put the conversation history single agent to the thread
user_prompt_3 = "waht is the latest news about google gcp?"
generate_conversation_thread_single(current_thread=block_list_thread, user_prompt=user_prompt_3)

# run agent based on the thread conversation history
run = project_client.agents.runs.create_and_process(thread_id=block_list_thread.id, agent_id=agent.id, temperature=0.1)

thread > created (id: thread_BQONwO6BIgd7dTxcGyZV6s0H)
message > created (id: msg_sailAXtMf1dzhFaUrr88DzhX)


In [14]:
block_list_agent_thread_message: ThreadMessage = project_client.agents.messages.get_last_message_by_role(thread_id = block_list_thread.id, role=MessageRole.AGENT)

display_message(block_list_agent_thread_message)

agent message text > I'm sorry, but I cannot assist with that request. (id: msg_uvD4bPxpqiL7UbR4GrFtDnKa)


### Define a Custom Event Handler
Here, we define a custom event handler to manage logs and outputs for debugging.
This handler will capture and display real-time events during the agent's operation.

In [15]:
class MyEventHandler(AgentEventHandler):
    def __init__(self):
        super().__init__()
        self._current_message_id = None
        self._accumulated_text = ""

    def on_message_delta(self, delta: MessageDeltaChunk) -> None:
        # If a new message id, start fresh
        if delta.id != self._current_message_id:
            # First, if we had an old message that wasn't completed, finish that line
            if self._current_message_id is not None:
                print()  # move to a new line
            
            self._current_message_id = delta.id
            self._accumulated_text = ""
            print("\nassistant > ", end="")  # prefix for new message

        # Accumulate partial text
        partial_text = ""
        if delta.delta.content:
            for chunk in delta.delta.content:
                # partial_text += chunk.text.get("value", "")
                if isinstance(chunk, MessageDeltaTextContent):
                    partial_text += chunk["text"].get("value", "")
                elif isinstance(chunk, MessageDeltaImageFileContent):
                    partial_text += chunk["image_file"].get("file_id", "")
        self._accumulated_text += partial_text

        # Print partial text with no newline
        print(partial_text, end="", flush=True)

    def on_thread_message(self, message: ThreadMessage) -> None:
        # When the assistant's entire message is "completed", print a final newline
        if message.status == "completed" and message.role == "assistant":
            print()  # done with this line
            self._current_message_id = None
            self._accumulated_text = ""
        else:
            # For other roles or statuses, you can log if you like:
            print(f"{message.status.name.lower()} (id: {message.id})")

    def on_thread_run(self, run: ThreadRun) -> None:
        print(f"status > {run.status.name.lower()}")
        if run.status == "failed":
            print(f"error > {run.last_error}")

    def on_run_step(self, step: RunStep) -> None:
        print(f"{step.type.name.lower()} > {step.status.name.lower()}")

    def on_run_step_delta(self, delta: RunStepDeltaChunk) -> None:
        # If partial tool calls come in, we log them
        if delta.delta.step_details and delta.delta.step_details.tool_calls:
            for tcall in delta.delta.step_details.tool_calls:
                if getattr(tcall, "function", None):
                    if tcall.function.name is not None:
                        print(f"tool call > {tcall.function.name}")

    def on_unhandled_event(self, event_type: str, event_data):
        print(f"unhandled > {event_type} > {event_data}")

    def on_error(self, data: str) -> None:
        print(f"error > {data}")

    def on_done(self) -> None:
        print("done")

### Implement the Main Chat Functions
These functions define how user messages and tool interactions are processed.
It uses the agent's thread to handle conversations and streams partial responses.

In [16]:
def extract_bing_query(request_url: str) -> str:
    """
    Extract the query string from something like:
      https://api.bing.microsoft.com/v7.0/search?q="latest news about Microsoft January 2025"
    Returns: latest news about Microsoft January 2025
    """
    match = re.search(r'q="([^"]+)"', request_url)
    if match:
        return match.group(1)
    # If no match, fall back to entire request_url
    return request_url

def extract_search_annotation(
        annotation: MessageDeltaTextUrlCitationAnnotation, text_value_str: str) -> str:
    """
    {'index': 0, 'type': 'text', 'text': {'value': '【3:1†Siemens fiscal report 2024】', 'annotations': [{'index': 0, 'type': 'url_citation', 'text': '【3:1†Siemens fiscal report 2024】', 'start_index': 1369, 'end_index': 1401, 'url_citation': {'url': 'doc_1', 'title': 'Siemens_Report_FY2024.pdf'}}]}}
    """
    if annotation["type"] == "url_citation":
        url = annotation["url_citation"]["url"]
        title = annotation["url_citation"].get("title", "")
        start_idx = annotation.get("start_index", 0)
        end_idx = annotation.get("end_index", 0)
        return f" [{text_value_str.strip()} {title}]({url}) ({start_idx}-{end_idx})"
    else:
        return ""

def convert_dict_to_chatmessage(msg: dict) -> ChatMessage:
    """
    Convert a legacy dict-based message to a gr.ChatMessage.
    Uses the 'metadata' sub-dict if present.
    """
    return ChatMessage(
        role=msg["role"],
        content=msg["content"],
        metadata=msg.get("metadata", None)
    )

def azure_enterprise_chat(user_message: str, history: List[dict]):
    """
    Accumulates partial function arguments into ChatMessage['content'], sets the
    corresponding tool bubble status from "pending" to "done" on completion,
    and also handles non-function calls like bing_grounding or file_search by appending a
    "pending" bubble. Then it moves them to "done" once tool calls complete.

    This function returns a list of ChatMessage objects directly (no dict conversion).
    Your Gradio Chatbot should be type="messages" to handle them properly.
    """

    # Convert existing history from dict to ChatMessage
    conversation = []
    for msg_dict in history:
        conversation.append(convert_dict_to_chatmessage(msg_dict))

    # Append the user's new message
    conversation.append(ChatMessage(role="user", content=user_message))

    # Immediately yield two outputs to clear the textbox
    yield conversation, ""

    # Post user message to the thread (for your back-end logic)
    project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )

    # Mappings for partial function calls
    call_id_for_index: Dict[int, str] = {}
    partial_calls_by_index: Dict[int, dict] = {}
    partial_calls_by_id: Dict[str, dict] = {}
    in_progress_tools: Dict[str, ChatMessage] = {}

    # Titles for tool bubbles
    function_titles = {
        # "fetch_weather": "☁️ fetching weather",
        "fetch_datetime": "🕒 fetching datetime",
        # "fetch_stock_price": "📈 fetching financial info",
        # "send_email": "✉️ sending mail",
        "file_search": "📄 searching docs",
        "bing_grounding": "🔍 searching bing",
        "azure_ai_search": "🔎 ai search private index",
    }

    def get_function_title(fn_name: str) -> str:
        return function_titles.get(fn_name, f"🛠 calling {fn_name}")

    def accumulate_args(storage: dict, name_chunk: str, arg_chunk: str):
        """Accumulates partial JSON data for a function call."""
        if name_chunk:
            storage["name"] += name_chunk
        if arg_chunk:
            storage["args"] += arg_chunk

    def finalize_tool_call(call_id: str):
        """Creates or updates the ChatMessage bubble for a function call."""
        if call_id not in partial_calls_by_id:
            return
        data = partial_calls_by_id[call_id]
        fn_name = data["name"].strip()
        fn_args = data["args"].strip()
        if not fn_name:
            return

        if call_id not in in_progress_tools:
            # Create a new bubble with status="pending"
            msg_obj = ChatMessage(
                role="assistant",
                content=fn_args or "",
                metadata={
                    "title": get_function_title(fn_name),
                    "status": "pending",
                    "id": f"tool-{call_id}"
                }
            )
            conversation.append(msg_obj)
            in_progress_tools[call_id] = msg_obj
        else:
            # Update existing bubble
            msg_obj = in_progress_tools[call_id]
            msg_obj.content = fn_args or ""
            msg_obj.metadata["title"] = get_function_title(fn_name)

    def upsert_tool_call(tcall: dict):
        """
        1) Check the call type
        2) If "function", gather partial name/args
        3) If "bing_grounding" or "file_search", show a pending bubble
        """
        t_type = tcall.get("type", "")
        call_id = tcall.get("id", None)

        # If call_id is None, generate a unique one (for parallel calls)
        if call_id is None:
            call_id = str(uuid.uuid4())
        
        # If call_id is None, generate a unique one (for parallel calls)
        if call_id is None:
            if t_type in ("file_search", "bing_grounding"):
                call_id = str(uuid.uuid4())
            elif t_type == "code_interpreter":
                call_id = "code_interpreter"
            elif t_type == "azure_ai_search":
                call_id = "azure_ai_search"
            else:
                call_id = "unknown_tool"

        # --- BING GROUNDING ---
        if t_type == "bing_grounding":
            request_url = tcall.get("bing_grounding", {}).get("requesturl", "")
            if not request_url.strip():
                return

            query_str = extract_bing_query(request_url)
            if not query_str.strip():
                return

            msg_obj = ChatMessage(
                role="assistant",
                content=query_str,
                metadata={
                    "title": get_function_title("bing_grounding"),
                    "status": "pending",
                    "id": f"tool-{call_id}" if call_id else "tool-noid"
                }
            )
            conversation.append(msg_obj)
            if call_id is not None:
                in_progress_tools[call_id] = msg_obj
            return

        # --- FILE SEARCH ---
        elif t_type == "file_search":
            msg_obj = ChatMessage(
                role="assistant",
                content="searching docs...",
                metadata={
                    "title": get_function_title("file_search"),
                    "status": "pending",
                    "id": f"tool-{call_id}" if call_id else "tool-noid"
                }
            )
            conversation.append(msg_obj)
            if call_id is not None:
                in_progress_tools[call_id] = msg_obj
            return
        

        # --- Azure AI SEARCH --- 
        elif t_type == "azure_ai_search":
            if call_id not in in_progress_tools:
                msg_obj = ChatMessage(
                    role="assistant",
                    content="searching private index...",
                    metadata={
                        "title": get_function_title("azure_ai_search"),
                        "status": "pending",
                        "id": f"tool-{call_id}" if call_id else "tool-noid"
                    }
                )
                conversation.append(msg_obj)
                in_progress_tools[call_id] = msg_obj
            return

        # --- NON-FUNCTION CALLS ---
        elif t_type != "function":
            return

        # --- FUNCTION CALL PARTIAL-ARGS ---
        index = tcall.get("index")
        new_call_id = call_id
        fn_data = tcall.get("function", {})
        name_chunk = fn_data.get("name", "")
        arg_chunk = fn_data.get("arguments", "")

        if new_call_id:
            call_id_for_index[index] = new_call_id

        call_id = call_id_for_index.get(index)
        if not call_id:
            # Accumulate partial
            if index not in partial_calls_by_index:
                partial_calls_by_index[index] = {"name": "", "args": ""}
            accumulate_args(partial_calls_by_index[index], name_chunk, arg_chunk)
            return

        if call_id not in partial_calls_by_id:
            partial_calls_by_id[call_id] = {"name": "", "args": ""}

        if index in partial_calls_by_index:
            old_data = partial_calls_by_index.pop(index)
            partial_calls_by_id[call_id]["name"] += old_data.get("name", "")
            partial_calls_by_id[call_id]["args"] += old_data.get("args", "")

        # Accumulate partial
        accumulate_args(partial_calls_by_id[call_id], name_chunk, arg_chunk)

        # Create/update the function bubble
        finalize_tool_call(call_id)

    # -- EVENT STREAMING --
    with project_client.agents.runs.stream(
        thread_id=thread.id,
        agent_id=agent.id,
        # assistant_id=agent.id,
        event_handler=MyEventHandler()  # the event handler handles console output
    ) as stream:
        # pulling the result from the stream manually
        for item in stream:
            event_type, event_data, *_ = item

            # Remove any None items that might have been appended
            conversation = [m for m in conversation if m is not None]

            # 1) Partial tool calls
            if event_type == "thread.run.step.delta":
                step_delta = event_data.get("delta", {}).get("step_details", {})
                if step_delta.get("type") == "tool_calls":
                    for tcall in step_delta.get("tool_calls", []):
                        upsert_tool_call(tcall)
                    yield conversation, ""

            # 2) run_step
            elif event_type == "run_step":
                step_type = event_data["type"]
                step_status = event_data["status"]

                # If tool calls are in progress, new or partial
                if step_type == "tool_calls" and step_status == "in_progress":
                    for tcall in event_data["step_details"].get("tool_calls", []):
                        upsert_tool_call(tcall)
                    yield conversation, ""

                elif step_type == "tool_calls" and step_status == "completed":
                    for cid, msg_obj in in_progress_tools.items():
                        msg_obj.metadata["status"] = "done"
                    in_progress_tools.clear()
                    partial_calls_by_id.clear()
                    partial_calls_by_index.clear()
                    call_id_for_index.clear()
                    yield conversation, ""

                elif step_type == "message_creation" and step_status == "in_progress":
                    msg_id = event_data["step_details"]["message_creation"].get("message_id")
                    if msg_id:
                        conversation.append(ChatMessage(role="assistant", content=""))
                    yield conversation, ""

                elif step_type == "message_creation" and step_status == "completed":
                    yield conversation, ""

            # 3) partial text from the assistant
            elif event_type == "thread.message.delta":
                agent_msg = ""
                for chunk in event_data["delta"]["content"]:
                    # print("chunk > ", chunk)
                    if isinstance(chunk, MessageDeltaTextContent):
                        # Safely get the text value
                        text_obj: MessageDeltaTextContentObject = chunk.get("text", {})
                        text_value_str = text_obj.get("value", "")
                        annotations = text_obj.get("annotations", None)
                        if annotations:
                            # Extract the URL citation if available
                            for annotation in annotations: 
                                agent_msg += extract_search_annotation(annotation, text_value_str)
                        else:
                            agent_msg += text_value_str
                    elif isinstance(chunk, MessageDeltaImageFileContent):
                        file_id = chunk["image_file"].get("file_id", "")
                        byte_stream = project_client.agents.files.get_content(file_id=file_id)
                        # Encode to base64
                        # Join all bytes from the iterator
                        image_bytes = b"".join(byte_stream)
                        b64_image = base64.b64encode(image_bytes).decode("utf-8")
                        # Use Markdown to display the image in Gradio
                        agent_msg += f"![image](data:image/png;base64,{b64_image})"  

                message_id = event_data["id"]

                # Try to find a matching assistant bubble
                matching_msg = None
                for msg in reversed(conversation):
                    if msg.metadata and msg.metadata.get("id") == message_id and msg.role == "assistant":
                        matching_msg = msg
                        break

                if matching_msg:
                    # Append newly streamed text
                    matching_msg.content += agent_msg
                else:
                    # Append to last assistant or create new
                    if (
                        not conversation
                        or conversation[-1].role != "assistant"
                        or (
                            conversation[-1].metadata
                            and str(conversation[-1].metadata.get("id", "")).startswith("tool-")
                        )
                    ):
                        conversation.append(ChatMessage(role="assistant", content=agent_msg))
                    else:
                        conversation[-1].content += agent_msg

                yield conversation, ""

            # 4) If entire assistant message is completed
            elif event_type == "thread.message":
                if event_data["role"] == "assistant" and event_data["status"] == "completed":
                    for cid, msg_obj in in_progress_tools.items():
                        msg_obj.metadata["status"] = "done"
                    in_progress_tools.clear()
                    partial_calls_by_id.clear()
                    partial_calls_by_index.clear()
                    call_id_for_index.clear()
                    yield conversation, ""

            # 5) Final done
            elif event_type == "thread.message.completed":
                for cid, msg_obj in in_progress_tools.items():
                    msg_obj.metadata["status"] = "done"
                in_progress_tools.clear()
                partial_calls_by_id.clear()
                partial_calls_by_index.clear()
                call_id_for_index.clear()
                yield conversation, ""
                break

    return conversation, ""

### Build a Gradio UI
Create a Gradio interface for interacting with the enterprise agent. 
Include a chatbot component and a text input box for user queries.

In [17]:
brand_theme = gr.themes.Default(
    primary_hue="blue",
    secondary_hue="blue",
    neutral_hue="gray",
    font=["Segoe UI", "Arial", "sans-serif"],
    font_mono=["Courier New", "monospace"],
    text_size="lg",
).set(
    button_primary_background_fill="#0f6cbd",
    button_primary_background_fill_hover="#115ea3",
    button_primary_background_fill_hover_dark="#4f52b2",
    button_primary_background_fill_dark="#5b5fc7",
    button_primary_text_color="#ffffff",
    button_secondary_background_fill="#e0e0e0",
    button_secondary_background_fill_hover="#c0c0c0",
    button_secondary_background_fill_hover_dark="#a0a0a0",
    button_secondary_text_color="#000000",
    body_background_fill="#f5f5f5",
    block_background_fill="#ffffff",
    body_text_color="#242424",
    body_text_color_subdued="#616161",
    block_border_color="#d1d1d1",
    block_border_color_dark="#333333",
    input_background_fill="#ffffff",
    input_border_color="#d1d1d1",
    input_border_color_focus="#0f6cbd",
)

with gr.Blocks(theme=brand_theme, css="footer {visibility: hidden;}", fill_height=True) as demo:

    def clear_thread():
        global thread
        thread = project_client.agents.threads.create()
        return []

    def on_example_clicked(evt: gr.SelectData):
        return evt.value["text"]  # Fill the textbox with that example text

    gr.HTML("<h1 style=\"text-align: center;\">Azure AI Agent Service</h1>")

    chatbot = gr.Chatbot(
        type="messages",
        examples=[
            # {"text": "What's my company's remote work policy?"},
            # {"text": "Check if it will rain tomorrow?"},
            # {"text": "How is Contoso's stock doing today?"},
            # {"text": "Show a summary of the all HR policy."},
            # {"text": "What is the date today?"},
            {"text": "What is the latest news about Microsoft?"},
            # {"text": "How does microsoft do quantum computing?"},
            # {"text": "what is the latest approach of microsoft to do quantum computing?"},
            {"text": "summarize siemens fiscal report 2024 from my company source"},
            # {"text": "Hight 5 insight regarding Microsoft Fiscal Report 2025 Q3."},
        ],
        show_label=False,
        scale=1,
    )

    textbox = gr.Textbox(
        show_label=False,
        lines=1,
        submit_btn=True,
    )

    # Populate textbox when an example is clicked
    chatbot.example_select(fn=on_example_clicked, inputs=None, outputs=textbox)

    # On submit: call azure_enterprise_chat, then clear the textbox
    (textbox
     .submit(
         fn=azure_enterprise_chat,
         inputs=[textbox, chatbot],
         outputs=[chatbot, textbox],
     )
     .then(
         fn=lambda: "",
         outputs=textbox,
     )
    )

    # A "Clear" button that resets the thread and the Chatbot
    chatbot.clear(fn=clear_thread, outputs=chatbot)

# Launch your Gradio app
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### (Optional) delete agent, thread, and vector store resources
Uncomment out the next cell block to delete the resources created in this notebook.

In [18]:
# from azure.identity import DefaultAzureCredential
# from azure.ai.projects import AIProjectClient
# import os

# credential = DefaultAzureCredential()
# project_client_delete = AIProjectClient(
#    credential=credential,
#    endpoint=os.environ.get("PROJECT_ENDPOINT")
# )

# try:
#    project_client_delete.agents.delete_agent(agent.id)
#    print("Agent deletion successful.")
#    project_client_delete.agents.threads.delete(thread.id)
#    print("Thread deletion successful.")
#    project_client_delete.agents.vector_stores.delete(vector_store_id)
#    print("Vector store deletion successful.")
#    print("All deletions succeeded.")
# except Exception as e:
#    print(f"Error during deletion: {e}")

In [19]:
# existing_stores = project_client_delete.agents.vector_stores.list()
# for store in existing_stores:
#     project_client_delete.agents.vector_stores.delete(store.id)